In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import v_measure_score

In [2]:
# caricamento dataset di addestramento

train = pd.read_csv("train_data.csv")

train.head()

Unnamed: 0            timestamp  vessel_id  latitude  longitude  speed  \
0        7295  2022-12-28 08:27:36          9  41.51630   16.55744    3.2   
1       56390  2023-10-02 01:24:37          9  41.32891   16.58438    2.4   
2       52691  2023-07-19 01:23:58          9  41.67343   16.54426    3.1   
3       66486  2023-11-13 18:16:23          9  41.71650   16.57164    3.4   
4       54545  2023-09-14 08:45:51          7  41.65605   16.41567    2.9   

   course  cluster_id  
0   323.0           7  
1   334.0           7  
2   348.0           7  
3   169.0           7  
4   169.0           7

In [3]:
# creazione dataset di input ed etichette di output
x_columns = ["vessel_id", "latitude", "longitude", "speed", "course"]
x = train[x_columns]
y = train[["cluster_id"]]

In [4]:
x.head()

vessel_id  latitude  longitude  speed  course
0          9  41.51630   16.55744    3.2   323.0
1          9  41.32891   16.58438    2.4   334.0
2          9  41.67343   16.54426    3.1   348.0
3          9  41.71650   16.57164    3.4   169.0
4          7  41.65605   16.41567    2.9   169.0

In [5]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86291 entries, 0 to 86290
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   vessel_id  86291 non-null  int64  
 1   latitude   86291 non-null  float64
 2   longitude  86291 non-null  float64
 3   speed      86291 non-null  float64
 4   course     86291 non-null  float64
dtypes: float64(4), int64(1)
memory usage: 3.3 MB


In [6]:
y.head()

cluster_id
0           7
1           7
2           7
3           7
4           7

In [7]:
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86291 entries, 0 to 86290
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   cluster_id  86291 non-null  int64
dtypes: int64(1)
memory usage: 674.3 KB


In [8]:
# creazione di un dataset di validazione per l'algoritmo di clusterizzazione
x_train, x_test, y_train, y_val = train_test_split(x, y, stratify=y, test_size=0.2, random_state=42)

In [9]:
y_train.value_counts()

cluster_id
7             60782
8              7579
10              505
9               166
dtype: int64

In [10]:
y_val.value_counts()

cluster_id
7             15196
8              1895
10              126
9                42
dtype: int64

In [11]:
# addestramento di un modello di clustering di base su parte dei dati di addestramento
kmeans = KMeans(n_clusters=4, random_state=42, n_init=100, max_iter=1000, tol=1e-5,).fit(x_train.values)

In [12]:
# clusterizzazione del modello sui dati di addestramento
y_train_pred = kmeans.predict(x_train.values)

In [13]:
# verifica distribuzione valori nei cluster trovati
unique, counts = np.unique(y_train_pred, return_counts=True)
print(dict(zip(unique, counts)))

{0: 24041, 1: 32636, 2: 6509, 3: 5846}


In [14]:
# clusterizzazione del modello sui dati di validazione
# (estratti dal dataset di addestramento)
y_val_pred = kmeans.predict(x_test.values)

In [15]:
# verifica distribuzione valori nei cluster trovati
unique, counts = np.unique(y_val_pred, return_counts=True)
print(dict(zip(unique, counts)))

{0: 6095, 1: 8156, 2: 1549, 3: 1459}


In [16]:
# in base alla distribuzione dei valori nei cluster trovati
# nel dataset di addestramento e validazione rispetto alla 
# distribuzione dei cluter del dataset originale, modifichiamo
# le atichette assegnate ai singoli cluster
y_val_pred[y_val_pred == 0] = 8
y_val_pred[y_val_pred == 1] = 7
y_val_pred[y_val_pred == 2] = 10
y_val_pred[y_val_pred == 3] = 9

In [17]:
# verifica del valore di V-measure e confronto tra risultati
# ottenuti sui dati di train e validation
print("train V-measure", v_measure_score(np.squeeze(y_train.values), y_train_pred))
print("validation V-measure", v_measure_score(np.squeeze(y_val.values), y_val_pred))

train V-measure 0.028004039321115358
validation V-measure 0.02628851473640516


In [18]:
# caricamento dataset di test, per la sottomissione dei risultati
test = pd.read_csv("test_data.csv")

test.head()

Unnamed: 0            timestamp  vessel_id  latitude  longitude  speed  \
0       88005  2024-02-01 04:29:18          9  41.73961   16.62330    3.6   
1       39993  2023-06-08 22:51:38          8  41.44730   16.63323    9.6   
2       30076  2023-04-26 17:00:57          9  42.22374   15.53183    3.2   
3       50262  2023-07-11 23:17:26          9  41.52628   16.51668    4.7   
4       17885  2023-02-23 08:20:38          8  41.52383   16.66625    3.3   

   course  
0   162.0  
1   180.0  
2   338.0  
3   120.0  
4   162.0

In [19]:
# clusterizzazione sui dati di test 
# tramite il modello addestrato sui dati di train

y_test_preds = kmeans.predict(test[x_columns])

c:\cloud\gitlab\fh_formazioneinterna\venvnew\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but KMeans was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [20]:
# correzione id cluster tramite l'associazione
# tra id cluster originale e id cluster trovato
# dall'algoritmo KMeans
y_test_preds[y_test_preds == 0] = 8
y_test_preds[y_test_preds == 1] = 7
y_test_preds[y_test_preds == 2] = 10
y_test_preds[y_test_preds == 3] = 9

In [ ]:
# aggiunta della colonna "cluster_id" al dataset di test
# (era una colonna mancante, rispetto al dataset di 
# addestramento)
test["cluster_id"] = y_test_preds

In [ ]:
# ordinamento del dataset per id di riga
test.sort_values("Unnamed: 0", inplace=True)

In [22]:
# selezione delle sole colonne "indice di riga" e id cluster
test = test[["Unnamed: 0", "cluster_id"]]

In [23]:
# visualizzazione dataset creato per la sottomissione 
# alla Data-Challenge 
test.head()

Unnamed: 0  cluster_id
3034           6           8
2324          13           8
3397          78           8
1208          83           8
126          100           8

In [24]:
# esportazione dei risultati in formato CSV
np.savetxt("submission.csv", y_test_preds.astype(np.int8), delimiter=",", fmt="%d")